In [1]:
import numpy as np
import matplotlib.pyplot as plt

import qutip as qt

import sim_utils as su

MHz = 1. # work in units of MHz
pi = np.pi

In [2]:
### system parameters
K = 2. * pi * 6. * MHz # Kerr nonlinearity
e_2 = 2. * pi * 15.5 * MHz # two-photon drive strength
n_th = 0.04 # population of n=1 Fock state due to thermal noise

### drive parameters
tau = 320.0e-3 / MHz # drive ramp time, ns

### simulation parameters
N = 10 # dimensionality of the Hilbert space, per resonator
tmax = 1. / MHz # go for 1 μs
dt = 1.0e-3 / MHz

### get the encoded basis states
psi_0, psi_1 = su.cat_states(K, e_2, N)

### define the Hamiltonian
H = [su.H_kerr(K, N), su.H_2ph(e_2, N, strength_factor=su.tanh_ramp(tau, 0.))]

In [3]:
t = np.arange(0., tmax, dt)

### initialize to the ground state
psi_init = (1. - n_th) * qt.basis(N, 0) + n_th * qt.basis(N, 1)
rho_init = psi_init * psi_init.dag()

result = qt.mesolve(H, rho_init, t, options=qt.Options(store_states=True))

In [ ]:
P0 = np.array([su.calculate_prob(rho, psi_0, N) for rho in result.states])

plt.plot(t, P0)
plt.plot(t, su.tanh_ramp(tau, 0.)(t, _))
print(P0[-1])